In [291]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


Extraction des fichier csv 

In [292]:
train__set_features = pd.read_csv("training_set_features.csv")
test = pd.read_csv("test_set_features.csv")
train__set_features.shape

train__set_labels = pd.read_csv("training_set_labels.csv")

y_h1n1 = train__set_labels['h1n1_vaccine']
y_seasonal = train__set_labels['seasonal_vaccine']


In [293]:
train = pd.concat([train__set_features, y_h1n1, y_seasonal], axis=1)


On va gerer les valeurs manquantes et les valeurs categoriels 

In [294]:
train.select_dtypes(include='object').isna().sum()

# train = train.drop(['employment_industry'], axis=1)
# la moitié de employement _industry et _occupation est manquante 
train['employment_occupation'].value_counts()

# essayer de creer catagorie big industrie // small industrie 

xtkaffoo    1778
mxkfnird    1509
emcorrxb    1270
cmhcxjea    1247
xgwztkwe    1082
hfxkjkmi     766
qxajmpny     548
xqwwgdyp     485
kldqjyjy     469
uqqtjvyb     452
tfqavkke     388
ukymxvdu     372
vlluhbov     354
oijqvulv     344
ccgxvspp     341
bxpfxfdn     331
haliazsg     296
rcertsgn     276
xzmlyyjv     248
dlvbwzss     227
hodpvpew     208
dcjcmpih     148
pvmttkik      98
Name: employment_occupation, dtype: int64

In [295]:
train = train.drop(["employment_industry", "employment_occupation"], axis=1)
test = test.drop(["employment_industry", "employment_occupation"], axis=1)

col = train.select_dtypes(exclude=(np.number)).columns

train = pd.get_dummies(train, columns=col)
test = pd.get_dummies(test, columns=col)



Creation des colonnes dans le cas ou la categorie n'était pas le test set 

In [296]:
for col in train.columns: 
    if not(col in test.columns):
        test[col] = pd.Series(np.zeros(test.shape[0]), name=col)


Estimation des valeurs manquantes grace aux KNN 


In [297]:

# essayer d'utiliser le test set pour les NA
 
from sklearn.neighbors import KNeighborsRegressor

def manage_na(X,na_col): 
    X = X.copy()
    numerical = X.select_dtypes(np.number)
    non_na_columns = numerical.loc[:,numerical.isna().sum() == 0].columns
    X_train = numerical.loc[numerical[na_col].isna() == False,non_na_columns]
    y_train = numerical.loc[numerical[na_col].isna() == False,na_col]
    X_na = numerical.loc[numerical[na_col].isna() == True, non_na_columns]
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    y_na = knn.predict(X_na)
    X.loc[X[na_col].isna() == True, na_col] = y_na
    
    return X 

In [298]:
cond = train.corr()['seasonal_vaccine'].apply(abs).sort_values() > 0.2

nulls = train[cond.index[cond]].isna().sum() > 0


for col in nulls[nulls == True].index:
    train = manage_na(train, col)

nulls_test = test.isna().sum() > 0

for col in nulls_test[nulls_test == True].index:
    test = manage_na(test, col)





In [299]:
nulls = train.isna().sum().sort_values() > 0
nulls.sort_values()


respondent_id                          False
hhs_geo_region_oxchjgsf                False
hhs_geo_region_qufhixun                False
census_msa_MSA, Not Principle  City    False
employment_status_Employed             False
                                       ...  
behavioral_wash_hands                   True
behavioral_face_mask                    True
chronic_med_condition                   True
behavioral_touch_face                   True
doctor_recc_h1n1                        True
Length: 64, dtype: bool

In [300]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split


X = train


params = {
    "n_estimators": np.arange(10, 13,1), 
    "max_depth": np.arange(10, 13, 1),
    "min_samples_leaf": np.arange(9,12 , 1), 
}
grid = GridSearchCV(RandomForestRegressor(), params, cv=5)

nulls = X.isna().sum() > 0 

for col in nulls[nulls == True].index:
    X = manage_na(X, col)


X = X.drop(["h1n1_vaccine", "seasonal_vaccine"], axis=1)
test = test.drop(["h1n1_vaccine", "seasonal_vaccine"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y_seasonal,  test_size=0.2, random_state=32,)

# grid.fit(X_train, y_train)
# grid.best_params_





In [301]:
rf = RandomForestRegressor(max_depth=13,min_samples_leaf=7, n_estimators=14)

rf.fit(X_train, y_train)

y_seasonal_predict = rf.predict(test)
y_seasonal

rf.score(X_test, y_test)
y_seasonal_predict.mean()
y_seasonal_predict

array([0.14077589, 0.04932592, 0.61913087, ..., 0.21734284, 0.36717659,
       0.439694  ])

In [302]:
X_train, X_test, y_train, y_test = train_test_split(X, y_h1n1,  test_size=0.2, random_state=32,)

rf = RandomForestRegressor(max_depth=9,min_samples_leaf=9, n_estimators=11)

rf.fit(X_train, y_train)

# grid.fit(X_train, y_train)
# print(grid.best_params_)



y_h1n1_predict = rf.predict(test)


rf.score(X_test, y_test)
y_h1n1_predict.mean()

result = pd.concat([test["respondent_id"], pd.Series(y_h1n1_predict, name="h1n1_vaccine"), pd.Series(y_seasonal_predict, name="seasonal_vaccine")], axis=1)
result.to_csv("result.csv", index=False)




In [303]:
sub_format = pd.read_csv("submission_format.csv")

sub_format['respondent_id'].head()

0    26707
1    26708
2    26709
3    26710
4    26711
Name: respondent_id, dtype: int64